In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset

#### Let's checkout out a diagnostic netCDF4 file

In [ ]:
file = '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_amsua_metop-a_ges.2020092200.nc4'

with Dataset(file, mode='r') as f:
    print(f)

#### Save some variables:

In [ ]:
with Dataset(file, mode='r') as f:
    latitude = f.variables['Latitude'][:]
    longitude = f.variables['Longitude'][:]
    omf_wo_bc = f.variables['Obs_Minus_Forecast_unadjusted'][:]
    omf_w_bc = f.variables['Obs_Minus_Forecast_adjusted'][:]

In [ ]:
latitude

#### Make a pandas dataframe

In [ ]:
data = {
    'latitude': latitude,
    'longitude': longitude,
    'omf_wo_bc': omf_wo_bc,
    'omf_w_bc': omf_w_bc
}

df = pd.DataFrame(data)
df

#### Thankfully, PyGSI has the ability to read in all the data into a clean dataframe in two lines:

In [ ]:
from pyGSI.diags import Radiance

# Create diag object
diag = Radiance(file)

# Create dataframe
df = diag.get_data()
df

#### Let's grab data from channel 11 and with qc marks set to 0

In [ ]:
df = diag.get_data(channel=[11], qcflag=[0])
df

Notice the difference between the two dataframes? PyGSI takes all the hassle out of sorting GSI diagnostics! Let's grab some data and work on some plots. We can list the variables by using the command here:

In [ ]:
df.columns

In [ ]:
latitude = df['latitude'].to_numpy()
longitude = df['longitude'].to_numpy()
hofx_wo_bc = df['hofx_unadjusted'].to_numpy()
obs = df['observation'].to_numpy()
omf_w_bc = df['omf_adjusted'].to_numpy()

### Plotting with EMCPy
Now that we have our data, let's make some plots using EMCPy. We will start by plotting observations vs. H(x) w/out bias correction on a scatter plot. First, let's create the EMCPy Scatter object.

In [ ]:
from emcpy.plots.plots import Scatter
from emcpy.plots.create_plots import CreatePlot

# Create Scatter object
sctr = Scatter(obs, hofx_wo_bc)
sctr

Above, we've made a new EMCPy Scatter object. Let's take a look at what that object looks like:

In [ ]:
sctr.__dict__

#### Create a simple scatter plot

In [ ]:
# Create Plot and draw data
myplt = CreatePlot()
plt_list = [sctr]
myplt.draw_data(plt_list)

Here we have a very basic scatter plot. But to someone else, they may not know what we are looking at. So let's add some features and customize the plot a bit.

#### Change some features for the Scatter data

In [ ]:
sctr.color = 'lightblue'
sctr.markersize = 2
sctr.vmin = 200
sctr.vmax = 225
# Add linear regression feature in scatter object
sctr.add_linear_regression()

In [ ]:
sctr.__dict__

#### Add some plot features:

In [ ]:
myplt = CreatePlot()
plt_list = [sctr]
myplt.draw_data(plt_list)

# Add Titles
title = 'H(x) vs Observation\nAMSUA metop-a Channel 10'
myplt.add_title(title, fontsize=14, loc='left')
myplt.add_title('YYYYMMDDHH', loc='right', fontweight='semibold')

# Create x and y labels
myplt.add_xlabel(xlabel='Observation')
myplt.add_ylabel(ylabel='H(x) no BC')

# Add grid
myplt.add_grid(linewidth=0.5, color='grey', linestyle='--')

# Set x and y limits
myplt.set_xlim([sctr.vmin, sctr.vmax])
myplt.set_ylim([sctr.vmin, sctr.vmax])

# Add a legend
myplt.add_legend(loc='upper left')

#### Return figure and save

In [ ]:
fig = myplt.return_figure()
fig.savefig('./sample_scatter_plot.png')

### Plot Spatial Map Plot

In [ ]:
from emcpy.plots.map_tools import Domain, MapProjection
from emcpy.plots.map_plots import MapScatter
from emcpy.plots.create_plots import CreateMap

# Create scatter plot on global domian
mymap = CreateMap(figsize=(12, 8),
                  domain=Domain('global'),
                  proj_obj=MapProjection('plcarr'))
# Add coastlines
mymap.add_features(['coastlines'])

# Create EMCPy object
scatterobj = MapScatter(latitude=latitude,
                        longitude=longitude,
                        data=omf_w_bc)
scatterobj.cmap = 'coolwarm'
scatterobj.vmin = -1
scatterobj.vmax = 1

# Draw data onto map
mymap.draw_data([scatterobj])

# Add plot features
mymap.add_colorbar(label='Brightness Temperature (K)',
                   label_fontsize=12, extend='both')
mymap.add_title(label='AMSUA metop-a OmF',
                loc='center', fontsize=16)
mymap.add_xlabel(xlabel='Longitude')
mymap.add_ylabel(ylabel='Latitude')

# Return figure and save
fig = myplt.return_figure()
fig.savefig('./sample_spatial_plot.png')